In [ ]:
# 🚀 MULTI-MODEL SERVING ENDPOINT (GIT VARIABLE DRIVEN)

import os
import json
import time
import yaml
import traceback
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import (
    EndpointCoreConfigInput,
    ServedEntityInput
)
import mlflow
from mlflow.tracking import MlflowClient

print("=" * 80)
print("🚀 MULTI-MODEL SERVING (GIT VARIABLE CONTROLLED)")
print("=" * 80)

# --------------------------------------------------
# 1️⃣ LOAD PIPELINE CONFIG
# --------------------------------------------------
try:
    with open("pipeline_config.yml", "r") as f:
        pipeline_cfg = yaml.safe_load(f)
    print("✅ pipeline_config.yml loaded")
except Exception as e:
    print("❌ Failed to load pipeline_config.yml")
    traceback.print_exc()
    raise e

# --------------------------------------------------
# 2️⃣ LOAD MODEL SERVING CONFIG FROM GIT VARIABLE
# --------------------------------------------------
MODEL_SERVING_CONFIG_RAW = os.getenv("MODEL_SERVING_CONFIG")

if not MODEL_SERVING_CONFIG_RAW:
    raise ValueError("❌ MODEL_SERVING_CONFIG Git variable not set")

MODEL_SERVING_CONFIG = json.loads(MODEL_SERVING_CONFIG_RAW)

print("\n📦 MODEL_SERVING_CONFIG:")
print(json.dumps(MODEL_SERVING_CONFIG, indent=2))

# --------------------------------------------------
# 3️⃣ INIT CLIENTS
# --------------------------------------------------
w = WorkspaceClient()
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

# --------------------------------------------------
# 4️⃣ BUILD SERVED ENTITIES (TRAFFIC SPLIT)
# --------------------------------------------------
served_entities = []

for model_key, cfg in MODEL_SERVING_CONFIG.items():

    if not cfg.get("active", False):
        print(f"⏭️ Skipping inactive model: {model_key}")
        continue

    served_entities.append(
        ServedEntityInput(
            entity_name=cfg["model_name"],
            entity_version=str(cfg["version"]),
            workload_size=pipeline_cfg["serving"]["workload_size"],
            scale_to_zero_enabled=pipeline_cfg["serving"]["scale_to_zero_enabled"],
            traffic_percentage=int(cfg["traffic"])
        )
    )

if not served_entities:
    raise ValueError("❌ No active models found in MODEL_SERVING_CONFIG")

# --------------------------------------------------
# 5️⃣ ENDPOINT NAME (SINGLE PROD ENDPOINT)
# --------------------------------------------------
BASE_NAME = pipeline_cfg["models"]["base_name"]
ENDPOINT_NAME = f"{BASE_NAME}-prod"

print(f"\n🚀 Serving Endpoint: {ENDPOINT_NAME}")

# --------------------------------------------------
# 6️⃣ CREATE OR UPDATE ENDPOINT
# --------------------------------------------------
def endpoint_exists(name):
    try:
        w.serving_endpoints.get(name=name)
        return True
    except:
        return False

try:
    if endpoint_exists(ENDPOINT_NAME):
        print("🔄 Updating existing endpoint...")
        w.serving_endpoints.update_config(
            name=ENDPOINT_NAME,
            served_entities=served_entities
        )
    else:
        print("➕ Creating new endpoint...")
        w.serving_endpoints.create(
            name=ENDPOINT_NAME,
            config=EndpointCoreConfigInput(
                served_entities=served_entities
            )
        )
except Exception as e:
    print("❌ Failed to deploy serving endpoint")
    traceback.print_exc()
    raise e

# --------------------------------------------------
# 7️⃣ WAIT UNTIL READY
# --------------------------------------------------
print("\n⏳ Waiting for endpoint to be READY...")
start = time.time()
timeout = pipeline_cfg["serving"]["deployment_timeout"]

while time.time() - start < timeout:
    ep = w.serving_endpoints.get(name=ENDPOINT_NAME)
    state = str(ep.state)

    if "READY" in state and "NOT_UPDATING" in state:
        print("✅ Endpoint is READY")
        break

    time.sleep(pipeline_cfg["serving"]["status_check_interval"])
else:
    raise TimeoutError("❌ Endpoint did not become READY in time")

# --------------------------------------------------
# 8️⃣ SUCCESS
# --------------------------------------------------
print("\n🎉 SERVING DEPLOYMENT SUCCESSFUL")
print("📊 Traffic Distribution:")
for s in served_entities:
    print(f"   • {s.entity_name} v{s.entity_version} → {s.traffic_percentage}%")

